# Week 7 - Logistic regression
by Colin Rundel & David Elliott & Ozan Evkaya

1. [Setup](#setup)

2. [Binary Logistic Regression](#RBH)

3. [Model refinement](#refine)

4. [Multi-class Example](#mclog)

5. [Regularization Example](#SKV)



This week we will be implementing the logistic regression techniques from this weeks lecture in python. We will start with the binary case and then look at some extensions of the logistic regression model

---

# 1. Setup <a id='setup'></a>

## 1.1. Uploading your data to the colab
This notebook will be saved in your google drive in a folder "Colab Notebooks" by default, you should be fairly familiar with this by now.

When you run this cell you will need to give colab permission to access files in your drive.

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/mlp/week-7')
!ls

Mounted at /content/drive
week07.ipynb  week-07.zip


We will now unzip the workshop materials and place them in a subdirectory "ws-material/"

In [2]:
if not os.path.exists('ws-material'):
  print('Unzipping materials...')
  !unzip week-07.zip -d ws-material
else:
  print("Directory already exists!")

os.chdir('ws-material')
!ls

Unzipping materials...
Archive:  week-07.zip
   creating: ws-material/week-07/
   creating: ws-material/week-07/Data/
  inflating: ws-material/week-07/Data/Default.csv  
   creating: ws-material/week-07/Images/
  inflating: ws-material/week-07/Images/pairplot.png  
  inflating: ws-material/week-07/Images/UoE_Horizontal_Logo_282_v1_160215.png  
   creating: ws-material/week-07/scripts/
  inflating: ws-material/week-07/scripts/create_widgets.js  
  inflating: ws-material/week-07/scripts/create_widgets.py  
  inflating: ws-material/week-07/scripts/environment.yml  
  inflating: ws-material/week-07/scripts/requirements.txt  
  inflating: ws-material/week-07/scripts/show_solutions.py  
week-07


__Reminder__ 

- You may need to restart the runtime several times in the workshop, but you will not need to re-upload or unzip files again. 

## 1.2 Packages

Now lets load in the packages you wil need for this workshop.

In [3]:
# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

# Other necessary packages
from sklearn.datasets import load_iris           # for the Iris data
from IPython.display import Image                # displaying .png images
from sklearn.preprocessing import StandardScaler # scaling features
from sklearn.preprocessing import LabelEncoder   # binary encoding
from sklearn.pipeline import Pipeline            # combining classifier steps
from sklearn.preprocessing import PolynomialFeatures # make PolynomialFeatures
import warnings # prevent warnings
from time import time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold #About randomized search over parameters
from scipy.stats.distributions import uniform, loguniform # About creating random C values for regularization

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.metrics import classification_report_imbalanced
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



##  1.3 Helper Functions

Below are two helper functions we will be using in this workshop.

In [4]:
# tidy the output into a dataframe
# We will use for getting the summary of cross-validation results in some parts
def tidy_scores(score_dict):
    df = pd.DataFrame(score_dict)
    df.loc['mean'] = df.mean()
    df.loc['sd'] = df.std()
    df.rename({"test_score":"val_score"}, axis=1, inplace=True)
    df.index.name = "fold"
    return df.round(2)


# this creates the matplotlib graph to make the confmat look nicer
# IT WORKS FOR ONLY BINARY CLASSIFICATION CASE !
def pretty_confusion_matrix(confmat, labels, title, labeling=False, highlight_indexes=[]):

    labels_list = [["TN", "FP"], ["FN", "TP"]]
    
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            if labeling:
                label = str(confmat[i, j])+" ("+labels_list[i][j]+")"
            else:
                label = confmat[i, j]
            
            
            if [i,j] in highlight_indexes:
                ax.text(x=j, y=i, s=label, va='center', ha='center',
                        weight = "bold", fontsize=18, color='#32618b')
            else:
                ax.text(x=j, y=i, s=label, va='center', ha='center')
       
    # change the labels
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ax.set_xticklabels(['']+[labels[0], labels[1]])
        ax.set_yticklabels(['']+[labels[0], labels[1]])

    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    ax.xaxis.set_label_position('top')
    plt.suptitle(title)
    plt.tight_layout()
    
    plt.show()

# For ROC curve plotting function

def roc_plot(threshold=0.5):
    i = (np.abs(roc.threshold - threshold)).idxmin()

    sns.lineplot(x='false positive rate', y='true positive rate', data=roc, ci=None)

    plt.plot([0,1],[0,1], 'k--', alpha=0.5) # 0-1 line 
    plt.plot(roc.iloc[i,0], roc.iloc[i,1], 'r.')

    plt.title("threshold = %.2f" % threshold)
    plt.show() 

# For iris data example 

def species_label(theta):
	if theta==0:
		return raw_data.target_names[0]
	if theta==1:
		return raw_data.target_names[1]
	if theta==2:
		return raw_data.target_names[2]

## 1.4 Data

The dataset consists of 10000 individuals and whether their credit card has defaulted or not. Below is the column description: The main aim is to build the model using Logistic Regression and predict the accuracy of it. The included columns in the data set are as follows:

* `default` - Whether the individual has defaulted

* `student` - Whether the individual is the student

* `balance` - The balance in the individual's account

* `income` - Income of an individual

We read the data into python using pandas.





In [8]:
df_default = pd.read_csv("./week-07/Data/Default.csv", index_col=0)

# for now lets just drop the student varible.
df_default = df_default.drop("student", axis=1)
df_default.head()

,default,balance,income
1,No,729.526495,44361.62507
2,No,817.180407,12106.13470
3,No,1073.549164,31767.13895
4,No,529.250605,35704.49394
5,No,785.655883,38463.49588


We will begin by doing explanatory data analysis to examine the data set itself

---

### 🚩 Exercise 1

Examine the structure of the data. Answer the following questions simply;

1. Do you observe anything of note from descriptive statistics that may influence our models?
2.  Are there any missing values? 
3. Look at how the varibles relate to each other (Response: Default, Predictors: Balance and Income) (Hint: Maybe some scatterplot and boxplot for each feature in terms of different response class will be useful)

---

**!!! Add your text solution here !!!**


---

### 🚩 Exercise 2

1. Lets now create our feature matrix and response varible.
2.  Split the data into training and test sets (**Is there anything you should try account for when splitting the data ?**) Use the test size as $10\%$ of the whole sample
3. Convert your response variable into the numerical format

Note: For data splitting consider the the additional argument inside of `train_test_split` function, stratify in terms of the response data set. Otherwise, without stratify argument. 

You can check the details from here 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html



---

**!!! Add your comments about the answer for Q.2 here !!!**



`

## 2.1 Logistic Regression

---

### 🚩 Exercise 3

Fit a binary logistic regression model to the training data by using both balance and income as features, without any regularization. State the the accuracy score of the model over the testing data

For the details of the function check out 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

## 2.2 Model Predictions

Unlike with our previous regression models, the fitted LogisticRegression objects provide multiple prediction methods. Specifically: predict which predicts the class label (either 0 or 1), predict_proba which predicts the class probabilities, and predict_log_proba which predicts the log probabilities of each class


---

### 🚩 Exercise 4

Based on the above fitted logistic regression model, calculate

1. the predictions of the class probabilities
2. Calculate the mean squared error, accuracy score, AUC score of your logistic regression model's predictions. The function `mean_squared_error` , `accuracy_score` and `roc_auc_score` from `sklearn.metrics` will be useful for this. See the functions documentation if you need [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

---

### 🚩 Exercise 5

Consider the pipeline for the logistic regression model simply. Run 5-fold Cross Validation for this calculation and use the helper function tidy_scores() created above for average score calculation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline  # combining classifier steps

# One can add more model in the dictionary
model_dict = {"log": LogisticRegression(random_state=42, penalty="none")}

#print(model_dict[model_name]) 

for model_name in model_dict:
    linear_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model_dict[model_name])])
        
scores = tidy_scores(cross_validate(linear_pipe, X_train, y_train, cv=5, return_train_score=True))
print(scores.loc[['mean', 'sd']])

## 2.3 Performance Metrics

A binary classifier can make two types of errors:

- Incorrectly assign an individual __who defaults__ to the __no default__ category.
- Incorrectly assign an individual who __does not default__ to the __default__ category.

While the overall error rate is low, the error rate among individuals who defaulted is very high. From the perspective of a credit card company that is trying to identify high-risk individuals, this error rate among individuals who default may well be unacceptable.

---

### 🚩 Exercise 6

Derive the confusion matrix for the logistic regression model with two predictors by following the above pipeline. Afterwards, derive the followings;

1. False Positive Rate (FPR)
2. True Positive Rate (Recall)
3. Precision
4. F1-score

without using any additional built-in function from any module (You need to recall the definitions from our notes) 

Note that,

$$
\text{FPR} = \frac{\text{FP}}{\text{FP}+ \text{TN}}
$$

$$
\text{Recall} = \frac{\text{TP}}{\text{TP}+ \text{FN}}
$$

$$
\text{Precision} = \frac{\text{TP}}{\text{TP}+ \text{FP}}
$$

$$
F1 = 2\left(\frac{Precision \times Recall}{Precision + Recall}\right)
$$

What can you say about the model performance in terms of Confusion Matrix and obtained quantities 

In [ ]:
log_pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(random_state=42))])

log_pipe.fit(X_train, y_train)

# use the first classifier to predict the based on testing data    


# get the confusion matrix as a numpy array


**!!! Add your comments about the model performance here !!!**


**F1-score**

- F1-score is a combination of Recall and Precision. 
- It is typically used when there is an __uneven class distribution__ due to a large number of True Negatives that you are not as focused on. 

$$
F1 = 2\left(\frac{PRE \times REC}{PRE + REC}\right)
$$

__Notes__

- Using `sklearn.metrics` this can be gained using `f1_score(y_true=y_val, y_pred=predictions)`

---

### 🚩 Exercise 7

1. Calculate the $F1$ score using *f1_score* function for the logistic regression model with two predictors and compare your result with your calculation obtained in Q4 in Exercise 5 above ! 
2. Draw ROC and Precision-Recall Curve for the same model. Explain the obtained graphical results and state the difference.  

---

**!!! Add your comments here !!!**


## 3.1 Model Refinement

For those coming from other programming / modeling languages this may be somewhat surprising, this strange default is probably the most common reason that results from sklearn might not immediately match results from other tools. This behavior can be explicitly controlled via the penalty argument. Note that if you did wish to include a penalty on the coefficients then just like with ridge or lasso it is necessary to tune this penalty parameter. However, `LogisticRegression` does not use alpha for this tuning parameter but instead uses $C$ which is the inverse of the alpha we have been using - i.e. smaller values of $C$ imply more regularization.

For our logistic regression model, lets start by looking at the effects of changing the regularization strength ($C$). Note that the default value is 
$C = 1.0$ in `LogisticRegression` function.


---

### 🚩 Exercise 8

1. By completing the following the following code, derive the related graph. Based on the graphical output, discuss the effects of changing the regularization strength( $C$ ) shortly. 

Note: You can look at the details of `plot_decision_regions` function but not obligatory to run the code !

In [ ]:
from mlxtend.plotting import plot_decision_regions
scatter_kwargs = {'edgecolor': None, 'alpha': 0.7}
contourf_kwargs = {'alpha': 0.2}
scatter_highlight_kwargs = {'s': 120, 'label': 'Validation data', 'alpha': 0.7}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    fig, axes = plt.subplots(figsize=(15,5), ncols=2, nrows = 2)
    axes = axes.flatten()
    
    for i, c in enumerate([0.01, 0.05, 1., 10.]):
        
        log_pipe = Pipeline([
            ("scaler", ______()),
            ("model", LogisticRegression(C = c, random_state=42))])

        log_pipe.fit(________, _______)
    
        plot_decision_regions(X_train, y_train, clf = log_pipe, 
                              legend = 2, X_highlight = X_val[y_val==1],
                              contourf_kwargs = contourf_kwargs,
                              scatter_kwargs = scatter_kwargs,
                              scatter_highlight_kwargs = scatter_highlight_kwargs,
                              ax = axes[i]
                             )
        
        axes[i].set_title("C = {}".format(c))
        
    plt.suptitle("Logistic Regression Decision Boundaries (Defaulted Validation Data Circled)")

## 3.1 Grid Search

Before we start searching over hyperparameters, its worth noting that some of the folds may not have the same distribution of the classes. This means we could get a validation score that may be a poor estimate of performance (for example we may have a fold with very few positive classes or more than usual). Therefore when doing our gridsearch/randomsearch, we should use a `StratifiedKFold` to ensure the distribution of classes in our folds reflects the distribution in the larger data.

You can check the details for `RandomizedSearchCV` function from here 

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [ ]:
from scipy.stats.distributions import uniform, loguniform

log_pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", LogisticRegression(C = c, random_state=42))])

C_list = []
pwr = -5
for i in range(11):
    C_list.append(2**pwr)
    pwr+=2
    
# specify parameters and distributions to sample from
log_param_dist = {'model__C':loguniform(C_list[0], C_list[-1])}

log_rs = RandomizedSearchCV(______, 
                            param_distributions = log_param_dist,
                            n_iter = 60, 
                            scoring = ["accuracy", "f1","recall","precision"], 
                            cv = StratifiedKFold(n_splits=5),
                            refit = "f1", 
                            random_state = 42,
                            return_train_score = True)

log_rs.fit(______, ______)

1. Convert your `log_rs.cv_results_` into pandas data frame and sort the values in terms "mean_test_accuracy" in descending order by using `sort_values` function !!!

2. Look at the first 6 values in this sorted values and make comments on test accuracy !!!

In [ ]:
# About the summary of best 6 models 
# Convert your log_rs.cv_results_ into pandas data frame and sort the values in terms 
# "mean_test_accuracy" in descending order by using sort_values !!!

# Look at the first 6 values in this sorted values and make comments on test accuracy !!!


##  3.2 Improving Models with Imbalances

There are a number of methods available to address imbalances in a dataset when we refine our models, such as:

1. Weighting the classes in the model during training,
2. Changing the training metric,
3. Resampling the data.

### 3.2.1. Weights

During model fitting we can assign a larger penalty to wrong predictions on the minority class.

The heuristic used for `class_weight="balanced"` in Scikit-Learn (0.23.1) is:

$$
\frac{n}{Nc \times \sum\limits^n_{i=1}I(y_i \in S)},
$$

where $n$ are the number of samples, $Nc$ the number of classes, $I$ is an indicator function, and $S$ contains the class elements.

---

### 🚩 Exercise 9

Conduct a gridsearch or randomsearch using the following code chunk, this time also looking at both `C` and `class_weight`. Has this improved performance? 

1. When you change your refit with other scoring alternatives rather than using "accuracy", what is the impact of it on the result (test recall)
2. Is there any improvement when n_splits = 10 ?    

In [ ]:
log_pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", LogisticRegression(C = c, random_state=42))])
    
# specify parameters and distributions to sample from
log_param_dist = {
    'model__C':loguniform(C_list[0], C_list[-1]),
    'model__class_weight': [None, "balanced"]
}

bal_log_rs = RandomizedSearchCV(log_pipe, 
                            param_distributions=log_param_dist,
                            n_iter=60, 
                            scoring = ["accuracy", "f1","recall","precision"], 
                            cv=StratifiedKFold(n_splits=5),
                            refit="accuracy", 
                            random_state=42,
                            return_train_score=True)

bal_log_rs.fit(X_train, y_train)

# To convert it as a data frame
bal_log_rs_df = pd.DataFrame(bal_log_rs.cv_results_)

#This is sorted for accuarcy, when you change your refit argument above, be careful about that
bal_log_rs_df.sort_values("mean_test_accuracy", ascending=False)[["param_model__class_weight", 
                                                                  "param_model__C", 
                                                                  "mean_test_accuracy", 
                                                                  "std_test_accuracy"]].head()

,param_model__class_weight,param_model__C,mean_test_accuracy,std_test_accuracy
0,None,5.620905,0.973951,0.001197
15,None,22774.162439,0.973951,0.001197
58,None,215.441061,0.973951,0.001197
53,None,10079.898638,0.973951,0.001197
52,None,4.498096,0.973951,0.001197


### 3.2.2 Changing Training/Validation Metric

__Optimising for Accuracy__ 

During hyperparamter cross-validation we are choosing the model with the best __overall accuracy__. 

This gives us a model with the smallest possible total number of misclassified observations, irrespective of which class the errors come from$^5$. 

ML algorithms typically optimize a reward or cost function computed as a sum over the training examples, the decision rule is likely going to be biased toward the majority class$^9$.

__Notes__

- _"In other words, the algorithm implicitly learns a model that optimizes the predictions based on the most abundant class in the dataset, in order to minimize the cost or  maximize the reward during training."_<sup>9</sup>.

- You can check the details of `classification_report` function from here 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

In [ ]:
# This is the classification report based on the accuracy metric
from sklearn.metrics import classification_report

print(pd.DataFrame(classification_report(y_val, 
                                   bal_log_rs.predict(X_val), 
                                   labels=None, 
                                   target_names=list(LE.classes_), 
                                   sample_weight=None, 
                                   digits=2, 
                                   output_dict=True)).round(2))

**Note**

Changing the metric for what is defined as the _"best model"_ can help us prioritise models that make particular errors.

For example, a credit card company might particularly wish to avoid incorrectly classifying an individual who will default, whereas incorrectly classifying an individual who will not default, though still to be avoided, is less problematic. 

In this case, __recall__ would therefore be a useful metric to use.

Rather than run another cross-validation again, provided that in `scoring` you used a list that contained "recall", we can just use our results data to pick the model with the best "recall" instead of accuracy.

---

### 🚩 Exercise 10

Conduct a the following function for the new metric, recall, and then derive a similar classification report that you observed above

In [ ]:
from sklearn.base import clone

# we refit the best accuracy model on all the training data
# so lets do that for the best other metric models
def manual_refit(input_model, X, y, gs, metric, disp_df=[]):
    output_model = clone(input_model)
    
    gs_df = pd.DataFrame(gs.cv_results_).sort_values("mean_test_"+metric, ascending=False)
    
    if disp_df:
        display(gs_df[disp_df].head())
    
    params = gs_df["params"].iloc[0]
    output_model = output_model.set_params(**params)
    output_model = output_model.fit(X, y)
    
    return output_model

rec_model = manual_refit(_______, ______, ______, bal_log_rs, "recall")

---

### 🚩 Exercise 11

Derive the confusion matrix for each model described above, `bal_log_rs` and `rec_model` using the confusion_matrix function. State the similarity and  differences of the obtained confusion matrices 

### 3.2.3. Resampling

We can change the distribution of the classes in our training data.

#### Under-Sampling

A fast way to balance the data is just to randomly select a subset of the data for each class so they have the number of datapoints found in the smallest class.

__Notes__

- `RandomUnderSampler` is part of the Imblearn package, which allows for a lot of techniques for working with imballanced data.
- There is a `resample` method in `scikit-learn` but Imblearn is a bit smoother to work with.

If you want to use a sampler in a model pipeline then you can use the pipeline from imblearn. Using a sampler in a pipeline ensure you wont be training and validating your data on a smaller/larger sample than normal and get unrepresentative results!

You can check some of the details for `RandomUnderSampler` from here 

https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html

If you understand the following mechanism, you can write the `RandomOverSampler` case below !!!

In [ ]:
from imblearn.pipeline import Pipeline as ImPipeline
from imblearn.under_sampling import RandomUnderSampler


log_pipe = ImPipeline([
    ("scaler", StandardScaler()),
    ("sampler", RandomUnderSampler(random_state=123)),
    ("model", LogisticRegression(random_state=42))])
    
# specify parameters and distributions to sample from
log_param_dist = {'model__C':loguniform(C_list[0], C_list[-1])}

us_log_rs = RandomizedSearchCV(log_pipe, 
                            param_distributions = log_param_dist,
                            n_iter = 60, 
                            scoring = ["accuracy", "f1","recall","precision"], 
                            cv = StratifiedKFold(n_splits=5),
                            refit = "recall", 
                            random_state = 42,
                            return_train_score = True)

us_log_rs.fit(X_train, y_train)

In [ ]:
us_log_rs_df = pd.DataFrame(us_log_rs.cv_results_)
us_log_rs_df.sort_values("mean_test_recall", ascending=False)[["param_model__C", 
                                                               "mean_test_recall", 
                                                               "std_test_recall"]].head()

#### Oversampling
Data can be oversampled easily by randomly sampling from minority classes with replacement to duplicate original samples. 

Note:
make sure to oversample after splitting the training and validation sets or you may "bleed" information into the validation sets of the model when trying to test a model

In-other-words, make sure it is in a pipeline!

---

### 🚩 Exercise 12

Create a similar pipeline for oversampling strategy for the imbalanced data. It is very similar what you observed above as in `log_pipe`

Hint: Consider now `from imblearn.over_sampling import RandomOverSampler` as a starting point.

After getting the results in a similar way, compare the performance in terms of undersampling or oversampling

---
# 4 Multi-class Logistic Regression

For this part, basically we will use the iris data, already available to use and it has nice properties in terms of class size 

In [ ]:
# First load the data here
from sklearn.datasets import load_iris

# Loading data
raw_data = load_iris()
data_desc = raw_data.DESCR
data = pd.DataFrame(raw_data.data, columns=raw_data.feature_names)

# Some information on data set 
data.head()
data.info()
data.describe()

Some Visual Inspection on the data set

In [ ]:
sns.set_style('whitegrid')
sns.pairplot(data, hue='species')
plt.show()

**!!! Add your comments on obtained  !!!**



Since we will be attempting to predict the species of flower which is a finite list of categorical options (ie. Setosa, Virginica, Versicolor), we need to assign a numerical value for each of the species within dummy variables. Dummy variables are variables that take binary (True=1, False=0) assignments based on every category in a given column. For example, our categories are Setosa, Virginica and Versicolor. We will have a column for each of these species and will assign a 1 or 0 for each row depending on what species they are. So a sample row representing Setosa, will have the Setosa column value of 1 and the Virginica and Versicolor values would be assigned as 0.

In [ ]:
# Consider species_label function as a helper 

# data = pd.DataFrame(raw_data.data, columns=raw_data.feature_names)
data['species'] = [species_label(theta) for theta in raw_data.target]
#data['species_id'] = raw_data.target
print(data.head())

# Class size information
print(data.species.value_counts())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   actual_setosa  actual_versicolor  actual_virginica species  
0              1                  0                 0  setosa  
1              1                  0                 0  setosa  
2              1                  0                 0  setosa  
3              1                  0                 0  setosa  
4              1                  0                 0  setosa  
setosa        50
versicolor    50
virginica     50
Name: species, dtype: int64


In [ ]:
#Prepare the data set

#Encoding Species columns (to numerical values)
data['species'] = data['species'].astype('category').cat.codes
print(data['species'])
#Feature & Target Selection
#features = data.select_dtypes('float').columns
#target = ['species']

# X = feature values, all the columns except the last column
X = data.iloc[:, :-1]
print(X.head())

# y = target values, last column of the data frame
y = data.iloc[:, -1]

print(y.head())

---

### 🚩 Exercise 13

1. Partition your data set into training and testing with %80-%20 ratio 
3. Create One-Vs-Rest Logistic Regression pipeline and train your model
4. Create Multinomial Logistic Regression pipeline and train your model

Under this method/strategy a multi-class classification dataset (in our case IRIS) is split into multiple binary classification chunks/datasets. Then a binary classifier (in our case SVC) is trained on each of the binary classification datasets and a prediction(s) is made using the model that has the most confidence value. The IRIS dataset will be split into individual datasets for each Species versus every other Species. Following are the details:

1. Binary Classification: setosa vs versicolor
2. Binary Classification: setosa vs virginica
3. Binary Classification: versicolor vs virginica

Side Not: The **OneVsRestClassifier** class is very easy to use and requires that a classifier that is to be used for binary classification be provided to the **OneVsRestClassifier** as an argument.

---

### 🚩 Exercise 14

1. Test the performance of one-vs-rest and multinomial logistic models
2. Derive the confusion matrix for each model

---

**!!! Add your comments on obtained  !!!**


---

## 5. Regularization on logistic regression

Note that regularized logistic regression is implemented using the ‘liblinear’ library, ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ solvers in scikits-learn. Note that regularization is applied by default as $L_2$.

So, to compare logistic regression without any regularization, play with the parameters of `LogisticRegression` function.

For this purpose, we will continue with the better model as a result of the Exercise 8 !!!


---

### 🚩 Exercise 15

1. Rebuild your models above (both one-vs-rest and multinomial logistic regression) without any regularization
2. Derive the confusion matrix for the obtained model

---

Add your text solution here

---

## 7. Competing the worksheet

At this point you have hopefully been able to complete all the preceeding exercises. Now 
is a good time to check the reproducibility of this document by restarting the notebook's
kernel and rerunning all cells in order.

Once that is done and you are happy with everything, you can generate your PDF and turn it in on gradescope under the `mlp-week07` assignment.